In [2]:
from SoccerNet.Downloader import SoccerNetDownloader

mySoccerNetDownloader=SoccerNetDownloader(LocalDirectory="Jersey number data and labels")

In [ ]:
mySoccerNetDownloader.password = "s0cc3rn3t"

mySoccerNetDownloader.downloadDataTask(task="jersey-2023", split=["train","test","challenge"]) 

In [2]:
!pip install ultralytics supervision lap

In [5]:
import os
import cv2
import shutil

# المسارات الأصلية
DATA_ROOT = "SoccerNet Data/tracking-2023"
SPLITS = {
    "train": "dataset",   # train → dataset/train
    "test": "dataset"     # test → dataset/val
}

# دالة تحدد الكلاس (0 لاعب - 1 كرة)
def get_class(obj_id):
    if obj_id == 0:
        return 1  # كرة
    return 0      # لاعب

def convert_split(split_name, split_path, out_split):
    print(f"\n=== Processing {split_name} split ===")

    out_images = os.path.join(SPLITS[split_name], "images", out_split)
    out_labels = os.path.join(SPLITS[split_name], "labels", out_split)
    os.makedirs(out_images, exist_ok=True)
    os.makedirs(out_labels, exist_ok=True)

    # نلف على كل sequences
    for seq in os.listdir(split_path):
        seq_path = os.path.join(split_path, seq)
        gt_path = os.path.join(seq_path, "gt/gt.txt")
        img_path = os.path.join(seq_path, "img1")

        if not os.path.exists(gt_path) or not os.path.exists(img_path):
            continue

        print(f"  → {seq}")

        # نقرأ صورة واحدة عشان نجيب الحجم
        img_files = sorted(os.listdir(img_path))
        if len(img_files) == 0:
            continue
        first_img = cv2.imread(os.path.join(img_path, img_files[0]))
        H, W = first_img.shape[:2]

        # نقرأ ملف gt.txt
        with open(gt_path, "r") as f:
            lines = f.readlines()

        # نخزن اللابلز لكل فريم
        frame_dict = {}
        for line in lines:
            parts = line.strip().split(",")
            frame, obj_id, x, y, w, h = map(int, parts[:6])

            # normalize
            x_center = (x + w / 2) / W
            y_center = (y + h / 2) / H
            w_norm = w / W
            h_norm = h / H

            class_id = get_class(obj_id)
            yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n"

            if frame not in frame_dict:
                frame_dict[frame] = []
            frame_dict[frame].append(yolo_line)

        # نكتب الصور واللابلز
        for idx, img_file in enumerate(img_files, start=1):
            src_img = os.path.join(img_path, img_file)
            dst_img = os.path.join(out_images, f"{seq}_{img_file}")
            dst_label = os.path.join(out_labels, f"{seq}_{img_file.replace('.jpg', '.txt')}")

            # نسخ الصورة
            shutil.copy(src_img, dst_img)

            # كتابة اللابل
            labels = frame_dict.get(idx, [])
            with open(dst_label, "w") as f:
                f.writelines(labels)


# 🟢 شغل على train → dataset/train
convert_split("train", os.path.join(DATA_ROOT, "train/train"), "train")

# 🟢 شغل على test → dataset/val
convert_split("test", os.path.join(DATA_ROOT, "test/test"), "val")

print("\n✅ Conversion Finished! YOLO dataset ready.")



=== Processing train split ===
  → SNMOT-060
  → SNMOT-061
  → SNMOT-062
  → SNMOT-063
  → SNMOT-064
  → SNMOT-065
  → SNMOT-066
  → SNMOT-067
  → SNMOT-068
  → SNMOT-069
  → SNMOT-070
  → SNMOT-071
  → SNMOT-072
  → SNMOT-073
  → SNMOT-074
  → SNMOT-075
  → SNMOT-076
  → SNMOT-077
  → SNMOT-097
  → SNMOT-098
  → SNMOT-099
  → SNMOT-100
  → SNMOT-101
  → SNMOT-102
  → SNMOT-103
  → SNMOT-104
  → SNMOT-105
  → SNMOT-106
  → SNMOT-107
  → SNMOT-108
  → SNMOT-109
  → SNMOT-110
  → SNMOT-111
  → SNMOT-112
  → SNMOT-113
  → SNMOT-114
  → SNMOT-115
  → SNMOT-151
  → SNMOT-152
  → SNMOT-153
  → SNMOT-154
  → SNMOT-155
  → SNMOT-156
  → SNMOT-157
  → SNMOT-158
  → SNMOT-159
  → SNMOT-160
  → SNMOT-161
  → SNMOT-162
  → SNMOT-163
  → SNMOT-164
  → SNMOT-165
  → SNMOT-166
  → SNMOT-167
  → SNMOT-168
  → SNMOT-169
  → SNMOT-170

=== Processing test split ===
  → SNMOT-116
  → SNMOT-117
  → SNMOT-118
  → SNMOT-119
  → SNMOT-120
  → SNMOT-121
  → SNMOT-122
  → SNMOT-123
  → SNMOT-124
  → SNMOT-125

In [1]:
!yolo train model=yolo11n.pt data=soccer.yaml epochs=100 imgsz=1280 batch=16

^C


In [ ]:
from ultralytics import YOLO

# تحميل الموديل المدرب
model = YOLO("runs/detect/train/weights/best.pt")

# تجربة على صورة
results = model.predict("dataset/images/val/xxx.jpg", save=True)

# تجربة على فيديو (يطلع ملف جديد بالنتيجة)
results = model.predict("match.mp4", save=True)


## Tracking


In [1]:
from ultralytics import YOLO
import cv2
import os

# ====== إعدادات ======
frames_dir = "SoccerNet Data/tracking-2023/train/train/SNMOT-060/img1"
output_video = "tracking_result.mp4"

# تحميل موديل YOLO
model = YOLO("yolo11n.pt")  # ممكن تبدل لموديل مخصص لكرة القدم لو عندك

# ترتيب الفريمات بالأسماء
frame_files = sorted(os.listdir(frames_dir))
if not frame_files:
    raise ValueError("❌ لم يتم العثور على أي فريمات في المسار المحدد!")

# قراءة أول فريم لمعرفة حجم الفيديو
sample_frame = cv2.imread(os.path.join(frames_dir, frame_files[0]))
h, w, _ = sample_frame.shape

# إعداد VideoWriter لحفظ النتيجة
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, 25.0, (w, h))

# تشغيل التتبع باستخدام StrongSORT
tracker = model.track(
    source=frames_dir,
    tracker="botsort.yaml",  # تتبع مع ReID
    conf=0.25,
    classes=[0, 32],  # 0 = person, 32 = sports ball
    stream=True,  # نخلي النتيجة تيجي frame by frame
    verbose=False
)

# ====== المعالجة ======
for result in tracker:
    frame = result.plot()  # رسم المربعات + IDs مباشرة
    out.write(frame)       # حفظ في الفيديو
    cv2.imshow("Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()
out.release()

print(f"✅ تم حفظ الفيديو النهائي في: {output_video}")


KeyboardInterrupt: 